# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import csv
import os

# Import Pretty Print
from pprint import pprint

# Import API key
from api_keys import g_key

# Configure the API Key for Google Maps
gmaps.configure(api_key=g_key)

# Output Path & File (CSV)
output_dir = os.path.join("..", "output_data")
output_data_file = os.path.join(output_dir, "cities.csv")

# Input Path & File (CSV)
input_dir = os.path.join("..", "input_data")
input_data_file = os.path.join(input_dir, "cities.csv")

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Does our intput file exist?
exists = os.path.isfile(input_data_file)
if exists == False:
    print(f"The input file '{input_data_file}' does not exist.  Please correct the problem and try again.")
else:
    df_cities = pd.read_csv(input_data_file)
    
    # Convert Unix Date to Readable DateTime
    df_cities["DateTime"] = pd.to_datetime(df_cities["DT"], unit="s")

    # Now, for a bit of formatting
    df_cities["Lat"] = df_cities["Lat"].astype(float)
    df_cities["Lat"] = df_cities["Lat"].astype(float)
    df_cities["Lon"] = df_cities["Lon"].astype(float)
    df_cities["Temp"] = df_cities["Temp"].astype(float)
    df_cities["Feels_Like"] = df_cities["Feels_Like"].astype(float)
    df_cities["Hi"] = df_cities["Hi"].astype(float)
    df_cities["Low"] = df_cities["Low"].astype(float)
    df_cities["Cloudiness"] = df_cities["Cloudiness"].astype(float)
    df_cities["Humidity"] = df_cities["Humidity"].astype(float)
    df_cities["Wind"] = df_cities["Wind"].astype(float)
    
    # Get the list of cities from our DataFrame.
    cities = df_cities["City"]

    # Print out the dataframe
    print(df_cities)
# end if

             City            DT Country    Lat     Lon   Temp   Color  \
0       saskylakh  1.579887e+09      RU  71.92  114.08 -27.94    blue   
1          castro  1.579887e+09      BR -24.79  -50.01  74.53  orange   
2        hermanus  1.579887e+09      ZA -34.42   19.23  73.00  orange   
3       cape town  1.579887e+09      ZA -33.93   18.42  69.94  orange   
4    puerto ayora  1.579887e+09      EC  -0.74  -90.35  82.71     red   
..            ...           ...     ...    ...     ...    ...     ...   
543    talcahuano  1.579887e+09      CL -36.72  -73.12  75.20  orange   
544         arman  1.579887e+09      RU  59.70  150.17 -14.73    blue   
545           bud  1.579887e+09      US  39.45  -86.18  41.74   green   
546   deputatskiy  1.579887e+09      RU  69.30  139.90 -36.53    blue   
547      espinosa  1.579887e+09      BR -14.93  -42.82  67.17  orange   

     Feels_Like     Hi    Low  Cloudiness           Weather  Humidity   Wind  \
0        -47.51 -27.94 -27.94       100.0  

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# https://www.storybench.org/how-to-build-a-heatmap-in-python/
figure_params = {
    "height": "600px",
    "width": "800px",
    "border": "1px solid grey",
    "padding": "1px", 
    "margin": "0 auto 0 auto"
}

# Create Heatmap
locations = df_cities[["Lat", "Lon"]]
weights = df_cities["Humidity"]
fig = gmaps.figure(layout=figure_params, center=(0,0), zoom_level=2) 

heatmap_layer = gmaps.heatmap_layer(locations, weights=weights) 
fig.add_layer(heatmap_layer)

# Display our heatmap
fig

Figure(layout=FigureLayout(border='1px solid grey', height='600px', margin='0 auto 0 auto', padding='1px', wid…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# What are the perfect conditions for my next vacation?
# I like hot temperatures with high humidity, just a few clouds and a slight breeze
df_ideal_cities = df_cities.loc[(df_cities["Temp"] <= 95)
                                  &(df_cities["Temp"] >= 80)
                                  &(df_cities["Humidity"] >= 65)
                                  &(df_cities["Cloudiness"] <= 20)
                                  &(df_cities["Wind"] <= 10)]

# print our dataframe of cities with the perfect weather conditions for my next 3 day vacation
df_ideal_cities

,City,DT,Country,Lat,Lon,Temp,Color,Feels_Like,Hi,Low,Cloudiness,Weather,Humidity,Wind,DateTime
10,lorengau,1.579887e+09,PG,-2.02,147.27,83.53,red,89.47,83.53,83.53,5.0,clear sky,70.0,5.53,2020-01-24 17:26:25
69,manavalakurichi,1.579887e+09,IN,8.13,77.30,82.40,red,91.13,82.40,82.40,20.0,haze,83.0,4.70,2020-01-24 17:28:00
108,georgetown,1.579887e+09,MY,5.41,100.34,81.75,red,88.41,84.00,80.01,20.0,few clouds,69.0,2.24,2020-01-24 17:28:22
114,axim,1.579887e+09,GH,4.87,-2.24,82.83,red,87.51,82.83,82.83,2.0,clear sky,73.0,8.32,2020-01-24 17:28:52
119,kavieng,1.579887e+09,PG,-2.57,150.80,83.28,red,87.69,83.28,83.28,0.0,clear sky,65.0,5.97,2020-01-24 17:25:30
357,san patricio,1.579887e+09,MX,19.22,-104.70,82.40,red,89.38,82.40,82.40,20.0,few clouds,74.0,4.23,2020-01-24 17:29:24
432,acapulco,1.579887e+09,MX,16.86,-99.89,84.20,red,88.81,84.20,84.20,20.0,few clouds,66.0,6.82,2020-01-24 17:32:42
535,quirinopolis,1.579887e+09,BR,-18.45,-50.45,81.05,red,86.23,81.05,81.05,16.0,moderate rain,82.0,9.22,2020-01-24 17:37:18


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# df_ideal_weather
# What hotel option do we have at these cities
hotel_df = pd.DataFrame({"Hotel_Name": [], 
                               "City": [], 
                               "Country": [], 
                               "Lat": [], 
                               "Lon": [], 
                               "Location": []
                              })

# Our base URL for google maps place search
base_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?key={g_key}&type=lodging&radius=5000&location="


for index, row in df_ideal_cities.iterrows():
    # The location for our hotel search (Lat, Lon)
    location = f"{row['Lat']}, {row['Lon']}"    

    try:
        # Let's call this bad boy
        response = requests.get(f"{base_url}{location}").json()

        hotel_df = hotel_df.append({"Hotel_Name": response["results"][0]["name"],
                                    "City": row["City"],
                                    "Country": row["Country"],
                                    "Lat": response["results"][0]["geometry"]["location"]["lat"],
                                    "Lon": response["results"][0]["geometry"]["location"]["lng"],
                                    "Location": location}, ignore_index=True)

        # Drop empty rows
        hotel_df = hotel_df.dropna()

        # Format out dataframe
        hotel_df["Lat"] = hotel_df["Lat"].astype(float)
        hotel_df["Lon"] = hotel_df["Lon"].astype(float)
        
    except(IndexError):
        print(f"There was an issue with the city '{row['City']}'")

# Display our results
hotel_df

,Hotel_Name,City,Country,Lat,Lon,Location
0,Lorengau Harbourside Hotel,lorengau,PG,-2.023093,147.270242,"-2.02, 147.27"
1,Udaya Boys Hostel,manavalakurichi,IN,8.136521,77.326138,"8.13, 77.3"
2,Cititel Penang,georgetown,MY,5.421232,100.333104,"5.41, 100.34"
3,AXIM BEACH RESORT AND SPA,axim,GH,4.867198,-2.233986,"4.87, -2.24"
4,Nusa Island Retreat,kavieng,PG,-2.582576,150.781292,"-2.57, 150.8"
5,Hotel Cabo Blanco,san patricio,MX,19.206905,-104.680036,"19.22, -104.7"
6,Crowne Plaza Acapulco,acapulco,MX,16.859863,-99.876439,"16.86, -99.89"
7,Rota Hotéis,quirinopolis,BR,-18.447742,-50.447400,"-18.45, -50.45"


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Lorengau Harbourside Hotel</dd>\n<dt>City</dt><dd>lorengau</dd>\n<dt>Country</dt><dd>PG</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Udaya Boys Hostel</dd>\n<dt>City</dt><dd>manavalakurichi</dd>\n<dt>Country</dt><dd>IN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Cititel Penang</dd>\n<dt>City</dt><dd>georgetown</dd>\n<dt>Country</dt><dd>MY</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>AXIM BEACH RESORT AND SPA</dd>\n<dt>City</dt><dd>axim</dd>\n<dt>Country</dt><dd>GH</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Nusa Island Retreat</dd>\n<dt>City</dt><dd>kavieng</dd>\n<dt>Country</dt><dd>PG</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Cabo Blanco</dd>\n<dt>City</dt><dd>san patricio</dd>\n<dt>Country</dt><dd>MX</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Crowne Plaza Acapulco</dd>\n<dt>City</dt><dd>acapulco</dd>\n<dt>Country</dt><dd>MX</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Rota Hotéis</dd>\n<dt>City</dt><dd>quirinopolis</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n'

In [7]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Hotel Map
fig

Figure(layout=FigureLayout(border='1px solid grey', height='600px', margin='0 auto 0 auto', padding='1px', wid…